In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk(r'C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (1).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (10).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (100).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (1000).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (101).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (102).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (103).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (104).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (105).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\Apple___Apple_scab\image (106).JPG
C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug\App

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
from skimage.measure import shannon_entropy
from pywt import dwt2
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

# dataset 
data_path = "C:/Users/Shrizon/Documents/Thesis/Sadaf/datasets/data/waug"

# Function to extract features using DWT, GLCM, and PCA
def extract_features(image):
    # Appling DWT (Discrete Wavelet Transform)
    coeffs = dwt2(image, 'bior1.3')
    features_dwt = np.concatenate([level.flatten() for level in coeffs[:-1]])  # Excluding the LL component

    # Computing Shannon Entropy as a substitute for GLCM
    gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    entropy = shannon_entropy(gray_img)
    features_glcm = np.array([entropy])

    # Appling PCA (Principal Component Analysis)
    flattened_image = image.flatten()
    # Set n_components to a value less than or equal to 3
    pca = PCA(n_components=min(3, flattened_image.size))
    features_pca = pca.fit_transform(flattened_image.reshape(-1, 3))

    # Combining and return the features
    return np.concatenate((features_dwt, features_glcm, features_pca.ravel()))

# Load images and extract features
X = []
y = []

# Limiting the number of images to process
max_images_per_class = 100

for class_name in os.listdir(data_path):
    class_path = os.path.join(data_path, class_name)
    images = os.listdir(class_path)
    
    # Shuffling the images to get a diverse set
    images = shuffle(images, random_state=42)
    
    for i, filename in enumerate(images):
        if i >= max_images_per_class:
            break
        
        img_path = os.path.join(class_path, filename)
        img = cv2.imread(img_path)
        
        # Checking if the image is successfully loaded
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Adjusting the size as needed
            features = extract_features(img)
            X.append(features)
            y.append(class_name)

# Converting lists to NumPy arrays
X = np.array(X)
y = np.array(y)

# Standardizing features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training SVM model
model = SVC(kernel='linear', C=1.0, gamma='scale')  # You can experiment with different values
model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = model.predict(X_test)

# Evaluating the SVM model
accuracy_svm = accuracy_score(y_test, y_pred)
precision_svm = precision_score(y_test, y_pred, average='weighted')
recall_svm = recall_score(y_test, y_pred, average='weighted')
f1_svm = f1_score(y_test, y_pred, average='weighted')

# Displaying evaluation metrics
print(f"Accuracy: {accuracy_svm:.2f}")
print(f"Precision: {precision_svm:.2f}")
print(f"Recall: {recall_svm:.2f}")
print(f"F1 Score: {f1_svm:.2f}")